In [1]:
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

## Conexión de la base de datos con SQL Server y lectura de tabla en específico en DataFrame

In [2]:
# CONEXION CON LA BASE DE DATOS DESDE SQL SERVER

server = 'DESKTOP-7FKVQ08\WORLDOFFICE17'
database = 'airlines'
username='soporte'
password    ='123'
driver = '{ODBC Driver 17 for SQL Server}' 

conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Conectar a la base de datos
conn = pyodbc.connect(conn_str)

# Leer datos de una tabla en un DataFrame de pandas
query = 'SELECT * FROM flights'
df = pd.read_sql(query, conn)

# Mostrar los datos
print(df.shape)

# Cerrar la conexión a la base de datos
#conn.close()



(33121, 10)


In [3]:
# Definir las tablas 
TABLE_NAMES = ['seats', 'flights', 'aircrafts_data', 'airports_data', 'boarding_passes', 'bookings', 'ticket_flights', 'tickets']

## Visualización de los primeros 5 registros de cada una de las tablas junto con su dimensión

In [4]:
try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    
    # Consulta para obtener la lista de tablas
    query_tables = """
    SELECT TABLE_NAME 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG = ?
    """
    tables = pd.read_sql(query_tables, conn, params=[database])
    
    # Mostrar las tablas encontradas
    print("Tablas en la base de datos:")
    print(tables)
   
    # Iterar sobre las tablas y mostrar los primeros registros de cada una
    
    for table in tables['TABLE_NAME']:
        query = f'SELECT * FROM {table}'
        df = pd.read_sql(query, conn)
        print(f"\nPrimeros registros de la tabla {table}:")
        print(df.head())
        print(f"Dimensiones de la tabla {table}: {df.shape}")
        print(df.columns.tolist())
        
        
except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)
finally:
    # Asegurarse de cerrar la conexión a la base de datos
    try:
        if conn:
            conn.close()
    except Exception as e:
        print("Ocurrió un error al cerrar la conexión:")
        print(e)

Tablas en la base de datos:
        TABLE_NAME
0         bookings
1          flights
2            seats
3   ticket_flights
4   aircrafts_data
5          tickets
6    airports_data
7  boarding_passes

Primeros registros de la tabla bookings:
                                            book_ref            book_date  \
0  00000F                                        ...  2017-07-05 03:12:00   
1  0002D8                                        ...  2017-08-07 21:40:00   
2  0002DB                                        ...  2017-07-29 06:30:00   
3  0002F3                                        ...  2017-07-10 05:31:00   
4  00034E                                        ...  2017-08-04 16:52:00   

  total_amount             ticket_no  
0  $265,700.00  5435838975            
1   $23,600.00  5435767874            
2  $101,500.00  5433986733            
3   $69,600.00  5433036153            
4   $73,300.00  5435653907            
Dimensiones de la tabla bookings: (262730, 4)
['book_ref', 'bo

## Tipo de Datos

In [7]:
# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    
    # Consulta para obtener los tipos de datos de todas las tablas
    query = """
    SELECT 
        TABLE_NAME, 
        COLUMN_NAME, 
        DATA_TYPE
    FROM 
        INFORMATION_SCHEMA.COLUMNS
    ORDER BY 
        TABLE_NAME, 
        COLUMN_NAME;
    """
    
    # Ejecutar la consulta y cargar los resultados en un DataFrame
    df = pd.read_sql(query, conn)
    
    # Mostrar los resultados
    print("Tipos de datos de las columnas en todas las tablas:")
    print(df)
    
except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)
finally:
    # Asegurarse de cerrar la conexión a la base de datos
    try:
        if conn:
            conn.close()
    except Exception as e:
        print("Ocurrió un error al cerrar la conexión:")
        print(e)

Tipos de datos de las columnas en todas las tablas:
         TABLE_NAME          COLUMN_NAME DATA_TYPE
0    aircrafts_data        aircraft_code      char
1    aircrafts_data                model   varchar
2    aircrafts_data                range   varchar
3     airports_data         airport_code      char
4     airports_data         airport_name   varchar
5     airports_data                 city   varchar
6     airports_data          coordinates   varchar
7     airports_data             timezone   varchar
8   boarding_passes          boarding_no   varchar
9   boarding_passes            flight_id       int
10  boarding_passes              seat_no   varchar
11  boarding_passes            ticket_no      char
12         bookings            book_date   varchar
13         bookings             book_ref      char
14         bookings            ticket_no      char
15         bookings         total_amount   varchar
16          flights       actual_arrival   varchar
17          flights     actual

## Datos faltantes

In [8]:
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Conectar a la base de datos
conn = pyodbc.connect(conn_str)

missing_counts = {}
    
    # Iterar sobre las tablas definidas y contar valores faltantes
for table_name in TABLE_NAMES:
        # Leer datos de la tabla actual en un DataFrame de pandas
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql(query, conn)
        
        # Contar valores faltantes en el DataFrame
        missing_values = df.isnull().sum().sum()
        
        # Guardar el resultado en el diccionario
        missing_counts[table_name] = missing_values
    
    # Mostrar los resultados
for table_name, count in missing_counts.items():
        print(f'Tabla: {table_name} - Valores Faltantes: {count}')
    

Tabla: seats - Valores Faltantes: 0
Tabla: flights - Valores Faltantes: 31776
Tabla: aircrafts_data - Valores Faltantes: 0
Tabla: airports_data - Valores Faltantes: 0
Tabla: boarding_passes - Valores Faltantes: 0
Tabla: bookings - Valores Faltantes: 5201
Tabla: ticket_flights - Valores Faltantes: 0
Tabla: tickets - Valores Faltantes: 0


## Datos duplicados

In [9]:
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Conectar a la base de datos
conn = pyodbc.connect(conn_str)
    
    # Crear un diccionario para almacenar los resultados
duplicate_counts = {}
    
    # Iterar sobre las tablas definidas y verificar duplicados
for table_name in TABLE_NAMES:
        # Leer datos de la tabla actual en un DataFrame de pandas
        query = f'SELECT * FROM {table_name}'
        df = pd.read_sql(query, conn)
        
        # Contar filas duplicadas en la tabla actual
        duplicate_count = df.duplicated(keep=False).sum()
        
        # Guardar el resultado en el diccionario
        duplicate_counts[table_name] = duplicate_count
    
    # Mostrar los resultados
for table_name, count in duplicate_counts.items():
        print(f'Tabla: {table_name} - Duplicados: {count}')


Tabla: seats - Duplicados: 0
Tabla: flights - Duplicados: 0
Tabla: aircrafts_data - Duplicados: 0
Tabla: airports_data - Duplicados: 0
Tabla: boarding_passes - Duplicados: 0
Tabla: bookings - Duplicados: 0
Tabla: ticket_flights - Duplicados: 0
Tabla: tickets - Duplicados: 0


## Transformar tipos de datos

- Tabla aircrafts:
columna range de tipo varchar a tipo entero

In [10]:
# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a la base de datos.")

except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)

try:
    # Consulta para alterar la tabla y cambiar el tipo de columna
    alter_query = """
    ALTER TABLE aircrafts_data
    ALTER COLUMN [range] INT  -- Cambiar a INT según el tipo de datos adecuado
    """
    
    # Ejecutar la consulta de alteración
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()

    print("Columna 'range' cambiada a tipo INT.")

except pyodbc.Error as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error en la consulta SQL:")
    print(e)
except Exception as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error general:")
    print(e)

    

Conexión exitosa a la base de datos.
Columna 'range' cambiada a tipo INT.


- Tabla bookings:
columna book_date de tipo varchar a tipo DATETIME

In [11]:


# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a la base de datos.")

except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)

try:
    # Consulta para alterar la tabla y cambiar el tipo de columna
    alter_query = """
    ALTER TABLE bookings
    ALTER COLUMN [book_date] DATETIME 
    """
    
    # Ejecutar la consulta de alteración
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()

    print("Columna 'book_date' cambiada a tipo DATETIME.")

except pyodbc.Error as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error en la consulta SQL:")
    print(e)
except Exception as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error general:")
    print(e)



Conexión exitosa a la base de datos.
Columna 'book_date' cambiada a tipo DATETIME.



- Tabla bookings:
columna total_amount de tipo varchar a tipo money

In [12]:
# Establecer la conexión
conn = pyodbc.connect(conn_str)

try:
    # Definir la consulta SQL para cambiar el tipo de dato
    alter_query = '''
    ALTER TABLE bookings
    ALTER COLUMN total_amount MONEY;
    '''

    # Ejecutar la consulta
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()
    print("Tipo de dato modificado exitosamente.")

except Exception as e:
    print(f"Ocurrió un error: {str(e)}")

finally:
    # Cerrar la conexión
    conn.close()

Tipo de dato modificado exitosamente.


- Tabla flights:  columna scheduled_departure de tipo varchar a tipo DATETIME

In [13]:
# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a la base de datos.")

except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)

try:
    # Consulta para alterar la tabla y cambiar el tipo de columna
    alter_query = """
    ALTER TABLE flights
    ALTER COLUMN [scheduled_departure] DATETIME  
    """
    
    # Ejecutar la consulta de alteración
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()

    print("Columna 'scheduled_departure' cambiada a tipo DATETIME.")

except pyodbc.Error as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error en la consulta SQL:")
    print(e)
except Exception as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error general:")
    print(e)


Conexión exitosa a la base de datos.
Columna 'scheduled_departure' cambiada a tipo DATETIME.


- Tabla flights: columna scheduled_arrival de tipo varchar a tipo DATETIME

In [14]:
# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a la base de datos.")

except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)

try:
    # Consulta para alterar la tabla y cambiar el tipo de columna
    alter_query = """
    ALTER TABLE flights
    ALTER COLUMN [scheduled_arrival] DATETIME  
    """
    
    # Ejecutar la consulta de alteración
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()

    print("Columna 'scheduled_arrival' cambiada a tipo DATETIME.")

except pyodbc.Error as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error en la consulta SQL:")
    print(e)
except Exception as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error general:")
    print(e)

Conexión exitosa a la base de datos.
Columna 'scheduled_arrival' cambiada a tipo DATETIME.


- Tabla flights: columna actual_departure de tipo varchar a tipo DATETIME

In [15]:
# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a la base de datos.")

except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)

try:
    # Consulta para alterar la tabla y cambiar el tipo de columna
    alter_query = """
    ALTER TABLE flights
    ALTER COLUMN [actual_departure] DATETIME  
    """
    
    # Ejecutar la consulta de alteración
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()

    print("Columna 'actual_departure' cambiada a tipo DATETIME.")

except pyodbc.Error as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error en la consulta SQL:")
    print(e)
except Exception as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error general:")
    print(e)

Conexión exitosa a la base de datos.
Columna 'actual_departure' cambiada a tipo DATETIME.


- Tabla flights: columna actual_arrival de tipo varchar a tipo DATETIME

In [16]:
# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a la base de datos.")

except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)

try:
    # Consulta para alterar la tabla y cambiar el tipo de columna
    alter_query = """
    ALTER TABLE flights
    ALTER COLUMN [actual_arrival] DATETIME  
    """
    
    # Ejecutar la consulta de alteración
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()

    print("Columna 'actual_arrival' cambiada a tipo DATETIME.")

except pyodbc.Error as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error en la consulta SQL:")
    print(e)
except Exception as e:
    conn.rollback()  # Revertir cambios en caso de error
    print("Ocurrió un error general:")
    print(e)

Conexión exitosa a la base de datos.
Columna 'actual_arrival' cambiada a tipo DATETIME.


- tabla Ticket_flights:  se hace cambio de tipo de dato varchar a tipo money en la columna amount 

In [17]:
# Establecer la conexión
conn = pyodbc.connect(conn_str)

try:
    # Definir la consulta SQL para cambiar el tipo de dato
    alter_query = '''
    ALTER TABLE ticket_flights
    ALTER COLUMN amount MONEY;
    '''

    # Ejecutar la consulta
    cursor = conn.cursor()
    cursor.execute(alter_query)
    conn.commit()
    print("Tipo de dato modificado exitosamente.")

except Exception as e:
    print(f"Ocurrió un error: {str(e)}")

finally:
    # Cerrar la conexión
    conn.close()

Tipo de dato modificado exitosamente.


## Verificar cambio de tipos de datos

In [18]:
# Crear la cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    
    # Consulta para obtener los tipos de datos de todas las tablas
    query = """
    SELECT 
        TABLE_NAME, 
        COLUMN_NAME, 
        DATA_TYPE
    FROM 
        INFORMATION_SCHEMA.COLUMNS
    ORDER BY 
        TABLE_NAME, 
        COLUMN_NAME;
    """
    
    # Ejecutar la consulta y cargar los resultados en un DataFrame
    df = pd.read_sql(query, conn)
    
    # Mostrar los resultados
    print("Tipos de datos de las columnas en todas las tablas:")
    print(df)
    
except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:")
    print(e)
except Exception as e:
    print("Ocurrió un error general:")
    print(e)
finally:
    # Asegurarse de cerrar la conexión a la base de datos
    try:
        if conn:
            conn.close()
    except Exception as e:
        print("Ocurrió un error al cerrar la conexión:")
        print(e)

Tipos de datos de las columnas en todas las tablas:
         TABLE_NAME          COLUMN_NAME DATA_TYPE
0    aircrafts_data        aircraft_code      char
1    aircrafts_data                model   varchar
2    aircrafts_data                range       int
3     airports_data         airport_code      char
4     airports_data         airport_name   varchar
5     airports_data                 city   varchar
6     airports_data          coordinates   varchar
7     airports_data             timezone   varchar
8   boarding_passes          boarding_no   varchar
9   boarding_passes            flight_id       int
10  boarding_passes              seat_no   varchar
11  boarding_passes            ticket_no      char
12         bookings            book_date  datetime
13         bookings             book_ref      char
14         bookings            ticket_no      char
15         bookings         total_amount     money
16          flights       actual_arrival  datetime
17          flights     actual

## Analisis Descriptivo 

In [19]:
# Cadena de conexión
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    
    # Obtener la lista de tablas
    tables = pd.read_sql("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG = ?", conn, params=[database])
    
    # Iterar sobre las tablas y aplicar describe() a cada una
    for table in tables['TABLE_NAME']:
        query = f'SELECT * FROM {table}'
        df = pd.read_sql(query, conn)
        print(f"\nAnálisis estadístico de la tabla {table}:")
        print(df.describe())
        
except pyodbc.Error as e:
    print("Ocurrió un error en la conexión a la base de datos:", e)
except Exception as e:
    print("Ocurrió un error general:", e)
finally:
    if conn:
        conn.close()


Análisis estadístico de la tabla bookings:
                           book_date  total_amount
count                         262730  2.627300e+05
mean   2017-07-24 01:13:43.674418688  7.902140e+04
min              2017-06-21 14:05:00  3.400000e+03
25%              2017-07-12 08:18:00  2.900000e+04
50%              2017-07-24 05:23:00  5.590000e+04
75%              2017-08-05 01:57:00  9.920000e+04
max              2017-08-15 18:00:00  1.204500e+06
std                              NaN  7.762317e+04

Análisis estadístico de la tabla flights:
          flight_id            scheduled_departure  \
count  33121.000000                          33121   
mean   16561.000000  2017-08-15 13:02:08.429093632   
min        1.000000            2017-07-16 01:50:00   
25%     8281.000000            2017-07-31 10:10:00   
50%    16561.000000            2017-08-15 12:45:00   
75%    24841.000000            2017-08-30 15:20:00   
max    33121.000000            2017-09-14 20:55:00   
std     9561.353469   